This script processes the GloFAS data that you downloaded in the previous script. It takes the forecasts in GRIB format, reads in our metadata and extracts the values from the forecasts at the gauging stations of interest. We then output these values in csv format so they can be used for analysis such as comparisons to observed data. 


We first import the relevant packages from our environment file.

In [ ]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

In the following cell we define our country of interest and home directory before setting up the paths to the relevant data. You should only have to change the directory variable here. An output directory to store the results will also be created. 

The metadata and forecast data will then be read in for processing.

In [ ]:
country = 'zimbabwe'  # Define country of interest
directory = '/Users/jamietowner/Documents/flood_aa_training/'  # Define directory

# Define paths to data 
forecast_data_directory = os.path.join(directory, country, "data/forecasts/glofas")
metadata_directory = os.path.join(directory, country, "data/metadata")

# Load the CSV file containing station information (i.e., station name, lat, lon)
station_info_file = "metadata_zim.csv"
station_info_path = os.path.join(metadata_directory, station_info_file)
station_info = pd.read_csv(station_info_path)

# Define the output directory for CSV files
output_directory = os.path.join(directory, country, "data/forecasts/glofas/stations")
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Get a list of all forecast GRIB files in the directory
forecast_files = [f for f in os.listdir(forecast_data_directory) if f.endswith(".grib")]

To check our data has been read in correctly we can execute the following cells

In [ ]:
station_info

In [ ]:
forecast_files

The rest of the script performs the hard work. It takes each forecast file in GRIB format and finds the relevant metadata from our station_info file (i.e., lat,lon,name) and uses this information to extract the river discharge values at the closest grid point to our station coordindates. Finally, each forecast is processed and the values are appended to a dataframe and are stored as a csv for each station and forecast. 

In [ ]:
# Initialize tqdm progress bar
pbar = tqdm(total=len(forecast_files), desc="Extracting Data")

# Loop over each forecast file
for forecast_file in forecast_files:
    # Extract the date from the filename
    forecast_date = forecast_file.split('_')[1].split('.')[0]  # Assuming GloFAS_YYYY_MM_DD.grib
    
    # Open the GRIB file
    grib_file_path = os.path.join(forecast_data_directory, forecast_file)
    ds = xr.open_dataset(grib_file_path, engine='cfgrib', filter_by_keys={'dataType': 'pf'})

    # Find the names of latitude and longitude in the dataset
    lat_name = 'latitude' if 'latitude' in ds.coords else 'lat'
    lon_name = 'longitude' if 'longitude' in ds.coords else 'lon'

    # Loop over each station in the metadata file
    for index, row in station_info.iterrows():
        point_name = row['station name']
        latitude = row['lat']
        longitude = row['lon']

        # Find the nearest latitude and longitude in the dataset using the correct names
        lat_index = ds[lat_name].sel(**{lat_name: latitude}, method='nearest')
        lon_index = ds[lon_name].sel(**{lon_name: longitude}, method='nearest')

        # Extract river discharge data for the nearest point and for all ensemble members
        data_at_point = ds['dis24'].sel(**{lat_name: lat_index, lon_name: lon_index})

        # Initialize a list to store data for all timesteps
        all_steps_data = []

        # Extract the date for each step
        steps = ds.step.values
        ensemble_count = len(ds['dis24'].coords['number'])

        for step in steps:
            # Extract data for this specific step
            step_data = data_at_point.sel(step=step)

            # Ensure step_data has the correct shape (should be [ensemble_count])
            if step_data.shape[0] != ensemble_count:
                print(f"Shape mismatch for {point_name} on step {step}: expected {ensemble_count}, got {step_data.shape[0]}")
                continue  # Skip if there's a mismatch

            # Reshape step_data for DataFrame creation
            step_data_reshaped = step_data.values[:, np.newaxis]  # Reshape to (10, 1)

            # Create a DataFrame for this step
            df_step = pd.DataFrame(step_data_reshaped.T, columns=[f"ensemble_{num}" for num in ds['dis24'].coords['number'].values])

            # Add the date for this step
            date_value = pd.to_datetime(forecast_date) + pd.to_timedelta(step, unit='h')  # Assuming 24-hour timesteps
            df_step['date'] = date_value

            # Append to the list of all steps data
            all_steps_data.append(df_step)

        # Concatenate all step DataFrames into a single DataFrame
        if all_steps_data:
            final_df = pd.concat(all_steps_data, ignore_index=True)

            # Rearrange columns to have date first
            final_df = final_df[['date'] + [col for col in final_df.columns if col != 'date']]

            # Format the date for the CSV file name
            formatted_date = pd.to_datetime(forecast_date).strftime('%Y_%m_%d')

            # Create the filename in the format: station_name_YYYY_MM_DD_glofas.csv
            csv_file_name = os.path.join(output_directory, f"{point_name}_{formatted_date}_glofas.csv")

            # Save the DataFrame to a CSV file
            final_df.to_csv(csv_file_name, index=False)

    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()